In [1]:
import pandas as pd
import re 
import numpy as np
from pyteomics import mass as pymass
from tqdm import tqdm
tqdm.pandas()


In [2]:
PXD002516 = pd.read_csv("/home/robbe/ionbot/Voor_vergelijking_OSE/From_scratch/PXD002516.tsv", sep="\t")

/tmp/ipykernel_3827650/3427772400.py:1: DtypeWarning: Columns (5,8,9,12,13,15,16,17,18,19,21,26,27,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  PXD002516 = pd.read_csv("/home/robbe/ionbot/Voor_vergelijking_OSE/From_scratch/PXD002516.tsv", sep="\t")


In [3]:
def Comet_raw_seq(string):
    only_alpha = ""
    if not pd.isna(string):
        for char in string:
            if char.isalpha():
                only_alpha += char
        return only_alpha
    else:
        return np.nan

In [4]:
PXD002516["Comet Raw Sequence"] = PXD002516["Comet Sequence"].progress_apply(Comet_raw_seq)

100%|██████████| 506643/506643 [00:00<00:00, 615582.49it/s]


In [5]:
def ionbot_mods_format(string):


    def splitatn(strng, sep, pos):
        strng = strng.split(sep)
        return sep.join(strng[:pos]), sep.join(strng[pos:])

    resultlist = []
    if pd.isna(string):
        return np.nan
    else:
        # listofmods = splitatn(string, "|", 2)
        listofmods = re.findall("[^|]+\|[^|]+", string)
        for mod in listofmods:
            if not mod == "":
                modnamematch = re.search(r"\](.*)\[", mod)
                if modnamematch:
                    modname = modnamematch.group(1)
                    resultlist.append(modname.lower())
                elif "->" in mod:
                    substitutionmatch = re.search(r"\|(.*)\[", mod)
                    if substitutionmatch:
                        resultlist.append(substitutionmatch.group(1).lower())
                    else:
                        resultlist.append("hiero")
                elif "semi_tryptic" in mod:
                    resultlist.append("semi_tryptic")
                else:
                    resultlist.append("hiero")

        return sorted(resultlist)



In [6]:
PXD002516["ionbot_mods_formatted"] = PXD002516["ionbot modifications"].progress_apply(ionbot_mods_format)


100%|██████████| 506643/506643 [00:01<00:00, 384117.56it/s]


In [7]:
PXD002516["all_fragger_mods"] = PXD002516["Fragger modifications"].astype(str) + "#" + PXD002516["Fragger unexpected modifications"].astype(str)


In [8]:
def fragger_mods_format(string, reformatdict):
    if string == "nan#nan":
        return np.nan

    else:
        resultlist = []

        expmod = string.split("#")[0]
        unexpmod = string.split("#")[1]

        expmodlist = expmod.split(",")
        for expmodif in expmodlist:
            if "15.9949" in expmodif:
                resultlist.append("oxidation")
            elif "57.0214" in expmodif:
                resultlist.append("carbamidomethyl")
            elif "42.0106" in expmodif:
                resultlist.append("acetyl")
        
        firstunex = unexpmod.split(";")[0]
        
        mod1 = re.search(r"Mod1: (.*) \(Peak", firstunex)
        mod2 = re.search(r"Mod2: (.*) \(Peak", firstunex)

        if mod1:
            modname = mod1.group(1)
            try:
                resultlist.append(reformatdict[modname])
            except KeyError:
                firstoption = modname.split("/")[0]
                try:
                    resultlist.append(reformatdict[firstoption])
                except KeyError:
                    pass
        if mod2:
            modname = mod2.group(1)
            try:
                resultlist.append(reformatdict[modname])
            except KeyError:
                firstoption = modname.split("/")[0]
                try:
                    resultlist.append(reformatdict[firstoption])
                except KeyError:
                    pass
    
    if resultlist == []:
        return np.nan
    
    else:
        return sorted(resultlist)
        

# 13C(57.0214), 24C(57.0214), 34C(57.0214)#Mod1: Dehydration/Pyro-glu from E (PeakApex: -18.0104, Theoretical: -18.0106)
#11C(57.0214)#Mod1: Unannotated mass-shift 1.0510 (PeakApex: 1.0510); Mod1: deuterated methyl ester, Mod2: reduction (PeakApex: 1.0404)


In [9]:
db = pymass.Unimod()
reformatdict = {}
for mod in db.mods:
    reformatdict[mod["full_name"]] = mod["title"].lower()

In [10]:
PXD002516["fragger_mods_formatted"] = PXD002516["all_fragger_mods"].progress_apply(fragger_mods_format, reformatdict=reformatdict)


100%|██████████| 506643/506643 [00:01<00:00, 328762.42it/s]


In [11]:
def pfind_mods_format(string):
    if not pd.isna(string):
        listofmods = string.split(";")

        resultlist = []

        for mod in listofmods:
            if not mod == "":
                modname = re.search(r",(.*)\[", mod)
                if modname:
                    resultlist.append(modname.group(1).lower())
            
        return sorted(resultlist)
    
    else:
        return np.nan

In [12]:
PXD002516["pfind_mods_formatted"] = PXD002516["pFind Modification"].progress_apply(pfind_mods_format)


100%|██████████| 506643/506643 [00:00<00:00, 510198.32it/s]


In [13]:
def comet_mods_format(string):
    moddict = {"162.05": "hexose", "203.08":"hexnac", "42.01":"acetyl", "0.98":"deamidated", "15.99":"oxidation", "14.02":"methyl", "58.01":["acetyl","oxidation"]}
    if not pd.isna(string):
        howmanyC = string.count("C")
        resultlist = []
        if howmanyC:
            for i in range(howmanyC):
                resultlist.append("carbamidomethyl")
        matchlist = re.findall(r"\[(\d+\.\d+)\]", string)
        if matchlist:
            for i in matchlist:
                if type(moddict[i]) == str:
                    resultlist.append(moddict[i])
                elif type(moddict[i]) == list:
                    resultlist.extend(moddict[i])
            return sorted(resultlist)
        elif resultlist:
            return sorted(resultlist)
        else:
            return np.nan
    else:
        return np.nan


In [14]:
PXD002516["comet_mods_formatted"] = PXD002516["Comet Sequence"].progress_apply(comet_mods_format)


100%|██████████| 506643/506643 [00:00<00:00, 534040.85it/s]


In [15]:
PXD002516.to_csv("PXD002516_formatted_2.tsv", sep="\t")